In [1]:
from glob import glob 
from datasets import Dataset, Audio
files = glob("./data/audios/train/*.mp3")[:3]


In [2]:
audio_dataset = Dataset.from_dict({"audio": files}).cast_column("audio", Audio())
audio_dataset = audio_dataset.cast_column("audio", Audio(sampling_rate=16000))
audio_dataset['audio']

[{'path': './data/audios/train/Kassel v. Consolidated Freightways Corporation of Delaware-450-662.mp3',
  'array': array([ 3.17839635e-12,  2.63955281e-11,  2.32569675e-11, ...,
         -2.66883362e-05, -3.03912238e-05, -2.01078656e-05]),
  'sampling_rate': 16000},
 {'path': './data/audios/train/Harte-Hanks Communications, Inc. v. Connaughton-491-657.mp3',
  'array': array([4.28948950e-13, 2.82365955e-13, 8.91341471e-13, ...,
         6.11488576e-06, 1.15077193e-04, 6.56439952e-05]),
  'sampling_rate': 16000},
 {'path': "./data/audios/train/Perry Education Association v. Perry Local Educators' Association-460-37.mp3",
  'array': array([-3.66026931e-11, -4.91774621e-10, -2.37404263e-10, ...,
         -4.62534255e-04, -3.60558683e-04,  0.00000000e+00]),
  'sampling_rate': 16000}]

In [3]:
from transformers import AutoProcessor, pipeline
model_id = "openai/whisper-large-v3-turbo"

processor = AutoProcessor.from_pretrained(
    model_id, language="english", task="transcribe"
)

In [4]:
def prepare_dataset(example):
    audio = example["audio"]
    """
    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        #text=example["sentence"],
    )
    """
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model_id,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor
    )
    # compute input length of audio sample in seconds
    #example["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    
    return example

In [5]:
pipe = pipeline(
        "automatic-speech-recognition",
        model=model_id,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor
    )
pipe(audio_dataset["audio"][0], return_timestamps=True, generate_kwargs={"language": "english"})

/home/labicquette/.local/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=english, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=english.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

In [ ]:
common_voice = audio_dataset.map(
    prepare_dataset, num_proc=1
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
from faster_whisper import WhisperModel
model_size = "large-v3"
model = WhisperModel(model_size, device="cpu", compute_type="int8")
segments, info = model.transcribe(files[0], beam_size=5)
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Detected language 'en' with probability 0.964060
[0.00s -> 6.56s]  Thank you, gentlemen. The case is submitted. We'll hear arguments next in Castle against Consolidated Freightways.
[30.00s -> 59.98s]  Thank you.
[60.00s -> 68.62s]  Mr. Solicitor General, you may elevate the lectern if you wish.
[70.26s -> 73.00s]  Thank you, Mr. Chief Justice, and may it please the Court.
[76.16s -> 89.82s]  Iowa appeals from decisions below, holding that a nondiscriminatory Iowa statute limiting the length of twin-trailered vehicles to 60 feet
[90.00s -> 93.00s]  unconstitutionally burdens interstate commerce.
[94.60s -> 102.68s]  At stake here, however, appears to be the authority of the states to regulate vehicle length at all.
[104.34s -> 111.36s]  The Iowa legislature has determined that the length of vehicles should be limited in the interest of safety.
[112.72s -> 116.06s]  Implicit in that determination is a factual premise.
[116.26s -> 118.90s]  As a matter of fact, this is your sole argument